In [1]:

import numpy as np
import xarray as xr

from topfarm.aep import AEP_load, AEP
from topfarm.wake_model import WakeModel
from topfarm.wind_farm import WindFarmLayout
from topfarm.wind_resource import WindResourceNodes

In [2]:
from topfarm.wind_resource import gwcfile_to_ds

In [3]:
from topfarm.wind_resource import WindResource

In [4]:
filename = 'wrf_HRI_55.489N_7.832E_70m.lib'

In [5]:
ds = gwcfile_to_ds(filename)
wr = WindResource(ds)

In [8]:
wfl_path = '/Users/pe/git/fw/jennifu/examples/hornsrev.yml'
wind_farm = WindFarmLayout(yml_path=wfl_path)

# =============================================================================
# STEP 3: Define wake model

NOJ = WakeModel()  # N.O. Jensen by default

# =============================================================================
# STEP 4: Calculate AEP

aep = AEP(wr, wind_farm, NOJ, ti=100.0)  # set up model
aep_gross, aep_net = aep.cal_AEP()  # calculate AEP
print(f'Gross AEP: {aep_gross.sum():.4e}')
print(f'Net AEP:   {aep_net.sum():.4e}')

Gross AEP: 7.1107e+10
Net AEP:   7.0580e+10


In [9]:
# =============================================================================
# STEP 3: Define wake model

GCL = WakeModel(wake_model='fort_gclm', wind_farm_yml=wfl_path)  # N.O. Jensen by default

# =============================================================================
# STEP 4: Calculate AEP

aep = AEP(wr, wind_farm, GCL, ti=100.0)  # set up model
aep_gross, aep_net = aep.cal_AEP()  # calculate AEP
print(f'Gross AEP: {aep_gross.sum():.4e}')
print(f'Net AEP:   {aep_net.sum():.4e}')

Exception: ('Strange position array', (2, 80))

In [ ]:
%debug

In [ ]:
aep.wf_design[:, 0].shape
#aep.wf_design[:, 1]
#aep.wf_design[:, 4]
#aep.wf_design[:, 3]
#aep.c
#aep.local_wd_ideal_ikl
#aep.local_Ct_ikl
#aep.turbulence_il

In [ ]:
aep.local_ws_ideal_ikl.shape

In [ ]:
aep.local_ws_ideal_ikl.shape

In [ ]:
(aep_gross.sum() - aep_net.sum())/aep_gross.sum() 

In [ ]:
aep.turbulence_il

In [ ]:
aep.wind_farm.u_pcurve[0].shape

In [ ]:
aep.wind_farm.get_power(0, 8.0)

In [ ]:
aep.local_power_real_ikl.sum()

In [ ]:
delta_ws = (aep.local_ws_ideal_ikl[:, 1:, 1:] -
                    aep.local_ws_ideal_ikl[:, :-1, 1:])

delta_wd = (aep.local_wd_ideal_ikl[:, 1:, 1:] -
            aep.local_wd_ideal_ikl[:, 1:, :-1])

pdf_array = (aep.local_pdf_ikl[:, 1:, 1:] +
             aep.local_pdf_ikl[:, :-1, 1:]) / 2

power_ideal_array = (aep.local_power_ideal_ikl[:, 1:, 1:] +
                     aep.local_power_ideal_ikl[:, :-1, 1:]) / 2

power_real_array = (aep.local_power_real_ikl[:, 1:, 1:] +
                    aep.local_power_real_ikl[:, :-1, 1:]) / 2

mean_power_ideal = np.sum(
    delta_ws * delta_wd * pdf_array * power_ideal_array, (1, 2))

mean_power_real = np.sum(
    delta_ws * delta_wd * pdf_array * power_real_array, (1, 2))

num_hrs_a_year = 8760
AEP_gross = num_hrs_a_year * mean_power_ideal * aep.availability
AEP_net = num_hrs_a_year * mean_power_real * aep.availability

In [ ]:
AEP_gross

In [ ]:
aep.local_pdf_ikl.sum()